 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

In [1]:
# Batch correction and t-SNE plots of haematopoietic data sets, as in Figure 3.
this.dir <- dirname(parent.frame(2)$ofile)
setwd(this.dir)

ERROR: Error in dirname(parent.frame(2)$ofile): нужен аргумент -- текстовый вектор


In [2]:
library(scran)
dir.create("results", showWarning=FALSE)

Loading required package: BiocParallel
Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, cbind, colMeans, colSums, colnames, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, lengths, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, rank, rbind, rowM

In [3]:
# Load the output of "preparedata.R".
load("logdataFandA_all.RData") 
colnames(logDataF3)[colnames(logDataF3)=="other"] <- "Unsorted"
colnames(logDataA3)[colnames(logDataA3)=="ERY"] <- "MEP"
raw.all <- cbind(logDataF3, logDataA3)
first.batch <- rep(c(TRUE, FALSE), c(ncol(logDataF3), ncol(logDataA3)))

In [4]:
# Adding colours.
#base.color <- "grey"
color.legendF <- c(MEP="orange", GMP="chartreuse4", CMP="magenta", 
                   HSPC="cyan", LTHSC="dodgerblue", MPP="blue", LMPP="light blue", Unsorted="grey")
colmatF <- col2rgb(color.legendF) 

colmatA <- colmatF + 100 # A lighter shade.
colmatA[colmatA > 255] <- 255

In [5]:
#colmatF<-colmatF+200
#colmatF[colmatF > 255] <- 255
color.legendA <- setNames(rgb(colmatA[1,], colmatA[2,], colmatA[3,], maxColorValue=255), names(color.legendF))
allcolors <- c(color.legendF[colnames(logDataF3)], color.legendA[colnames(logDataA3)])
batch<-c( rep(1,ncol(logDataF3)),rep(2,ncol(logDataA3)) )

In [6]:
# Only keeping common cell types for PCA.
celltypes <- c(colnames(logDataF3), colnames(logDataA3))
pca.retain <- celltypes %in% c("MEP", "GMP", "CMP") 

In [7]:
# Making a plotting function.
plotFUN <- function(fname, Y, subset=NULL, ..., xlab="tSNE 1",ylab="tSNE 2",main="") {
  if (is.null(subset)) {
    subset <- seq_len(nrow(Y))
  }
  png(fname,width=900,height=700)
  par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
  plot(Y[,1], Y[,2], cex=2,
       pch=ifelse(first.batch, 21, 1)[subset], 
       col=ifelse(first.batch, "black", allcolors)[subset],
       bg=allcolors[subset], xlab=xlab, ylab=ylab, main=main) 
    dev.off()
}

In [8]:
batchcolor=c("lavender","lightcoral")
plotFUNb <- function(fname, Y, subset=NULL, ...) {
  if (is.null(subset)) {
    subset <- seq_len(nrow(Y))
  }
  png(fname,width=900,height=700)
  par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
  plot(Y[,1], Y[,2], cex=2,
       pch=ifelse(first.batch, 21, 1)[subset], 
       col=ifelse(first.batch, "black", batchcolor[batch[subset]]),
       bg=batchcolor[batch[subset]], ...)#,  xlab="tSNE 1",ylab="tSNE 2")
  dev.off()
}

In [9]:
######################
# No correction.

X.unc <- raw.all
t.unc <- t(X.unc)

In [1]:
library(Rtsne)

In [10]:
## Generating a t-SNE plot.
#require(Rtsne)
set.seed(0)
all.dists.unc <- as.matrix(dist(t.unc))
tsne.unc <- Rtsne(all.dists.unc, is_distance=TRUE, perplexity = 90)
plotFUN("results/uncFA.png", tsne.unc$Y, main="Uncorrected",  xlab="tSNE 1",ylab="tSNE 2")
plotFUNb("results/uncFAb.png", tsne.unc$Y, main="Uncorrected",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2

pdf 
  2

In [11]:
rm(all.dists.unc)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3782303,202.0,5684620,303.6,4703850,251.3
Vcells,52046379,397.1,153117260,1168.2,139288639,1062.7


In [12]:
# Generating a PCA plot.
pca.unc <- prcomp(t.unc[pca.retain,], rank=2)
pca.unc$x[ (pca.unc$x<(-0.08))]<- (-0.08)
plotFUN("results/pca_raw.png", pca.unc$x, subset=pca.retain, main="Uncorrected", ylim=c(-0.1, max(pca.unc$x[,2])),  xlab="PC 1",ylab="PC 2")

pdf 
  2

In [13]:
rm(t.unc)
gc()
######################## 

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3788127,202.4,5684620,303.6,4703850,251.3
Vcells,35866323,273.7,122493808,934.6,151056250,1152.5


In [14]:
install.packages('pracma')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [15]:
library(pracma)

In [16]:
# Performing the correction with MNN (turned down the sigma to improve mixing).

mnn.out<-mnnCorrect(logDataF3, logDataA3,k=20, sigma=0.1,cos.norm.in=TRUE, cos.norm.out=TRUE, var.adj=TRUE,compute.angle=TRUE)

X.mnn <- cbind(mnn.out$corrected[[1]], mnn.out$corrected[[2]])
t.mnn <- t(X.mnn)

In [17]:
# plot histogram of angles between batch vectors and 2 svds of the reference batch.
png(file="results/angles.png",width=900,height=700)
par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
hist(mnn.out$angles[[2]],xlab="Angle",ylab="Frequency",main="")
dev.off()

pdf 
  2

In [18]:
# Generating a t-SNE plot.
set.seed(0)
all.dists.mnn <- as.matrix(dist(t.mnn))
tsne.mnn <- Rtsne(all.dists.mnn, is_distance=TRUE, perplexity = 90)
plotFUN("results/mnnFA.png", tsne.mnn$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")
plotFUNb("results/mnnFAb3.png", tsne.mnn$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2

pdf 
  2

In [19]:
set.seed(0)
tsne.mnn2 <- Rtsne(t.mnn, perplexity = 90)
plotFUN("results/mnnFA_conventsne.png", tsne.mnn2$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2

In [20]:
rm(all.dists.mnn)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3826591,204.4,5684620,303.6,5684620,303.6
Vcells,84833130,647.3,211960498,1617.2,203634455,1553.7


In [21]:
# Generating a PCA plot.
pca.mnn <- prcomp(t.mnn[pca.retain,], rank=2)
#pca.mnn$x[ (pca.mnn$x<(-0.08))]<- (-0.08)
plotFUN("results/pca_mnn.png", pca.mnn$x, subset=pca.retain, main="MNN", ylim=c(-0.08,0.05),  xlab="PC 1",ylab="PC 2")

pdf 
  2

In [23]:
install.packages("destiny")

Warning message:
“package ‘destiny’ is not available (for R version 3.4.3)”

In [24]:
# Generating diffusion map plots.
library(destiny)
dm<-DiffusionMap(t.mnn,n_local = 150)
plotFUN("results/mnnFAdm12.png", dm@eigenvectors[,1:2], main="MNN",  xlab="DC 1",ylab="DC 2")
plotFUN("results/mnnFAdm23.png", cbind(dm@eigenvectors[,2],dm@eigenvectors[,3]), main="MNN",  xlab="DC 2",ylab="DC 4")
plotFUN("results/mnnFAdm13.png", cbind(dm@eigenvectors[,1],dm@eigenvectors[,3]), main="MNN",  xlab="DC 2",ylab="DC 4")

ERROR: Error in library(destiny): there is no package called ‘destiny’


In [ ]:
rm(t.mnn)
gc()

In [ ]:
######################## 
# Performing the correction with limma.

library(limma)
X.lm <- removeBatchEffect(raw.all, factor(first.batch))
t.lm <- t(X.lm)

In [ ]:
## Generating a t-SNE plot.
set.seed(0)
all.dists.lm <- as.matrix(dist(t.lm))
tsne.lm <- Rtsne(all.dists.lm, is_distance=TRUE, perplexity = 90)
plotFUN("results/lmfitFA.png", tsne.lm$Y, main="limma",  xlab="tSNE 1",ylab="tSNE 2")
plotFUNb("results/lmfitFAb.png", tsne.lm$Y, main="limma",  xlab="tSNE 1",ylab="tSNE 2")

In [ ]:
rm(all.dists.lm)
gc()

In [ ]:
# Generating a PCA plot.
pca.lm <- prcomp(t.lm[pca.retain,], rank=2)
pca.lm$x[ (pca.lm$x[,2] > (0.08)),2]<- (0.08)

plotFUN("results/pca_lm.png", pca.lm$x, subset=pca.retain, main="limma", ylim=c(min(pca.lm$x[,2]), 0.05),  xlab="PC 1",ylab="PC 2")

In [ ]:
rm(t.lm)
gc()

In [ ]:
######################## 
# Performing the correction with ComBat.

library(sva)
Z <- factor(first.batch)
X.combat <- ComBat(raw.all,Z,mod=NULL,prior.plots = FALSE)
t.combat <- t(X.combat)

In [ ]:
## Generating a t-SNE plot.
set.seed(0)
all.dists.combat <- as.matrix(dist(t.combat))
tsne.combat<-Rtsne(all.dists.combat, is_distance=TRUE,perplexity = 90)
plotFUN("results/combatFA.png", tsne.combat$Y, main="ComBat",  xlab="tSNE 1",ylab="tSNE 2")
plotFUNb("results/combatFAb.png", tsne.combat$Y, main="ComBat",  xlab="tSNE 1",ylab="tSNE 2")

In [ ]:
rm(all.dists.combat)
gc()

In [ ]:
# Generating a PCA plot.
pca.combat <- prcomp(t.combat[pca.retain,], rank=2)
plotFUN("results/pca_com.png", pca.combat$x, subset=pca.retain, main="ComBat", ylim=c(min(pca.combat$x[,2]), 0.05),  xlab="PC 1",ylab="PC 2")

In [ ]:
rm(t.combat)
gc()

In [ ]:
######################## 
# Making the legend (using PDF for better resolution).

pdf(file="results/legend.pdf", width=7, height=7)
plot(0,0,type="n", bty="n", axes=FALSE, xlab="", ylab="")
legend(x=-1, y=1, legend=names(color.legendF), pch=21, cex=2.5, col="black", pt.bg=color.legendF, bty="n")
legend(x=0, y=1, legend=names(color.legendA)[1:3], pch=21, cex=2.5, col="black", pt.bg=color.legendA[1:3], bty="n")
dev.off()

######################## 
# END

In [ ]:
object.size()

In [29]:
!head unitab_amit.txt

ERROR: Error in parse(text = x, srcfile = src): <text>:1:7: неожиданный симфол
1: !head unitab_amit.txt
          ^
